In [1]:
import model_lib
import numpy as np
import warnings

warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 16
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 16
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
with open(train_pickle,"rb") as train_ann:
    train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [6]:
import torch.optim as optim
import torch
import torch.nn.functional as F
import torch.nn as nn
net = model_lib.MultiHGModel()


In [7]:
# net.vgg0.load_state_dict(torch.load("./models/split_vgg16_features_1.pt"))
# net.vgg1.load_state_dict(torch.load("./models/split_vgg16_features_4.pt"))
pretrained_dict = torch.load(model_dir+"bbox_01.pt")
net_dict = net.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}

net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

In [8]:
# net.vgg1.load_state_dict(net.vgg0.state_dict())
# net.mp1.load_state_dict(net.mp0.state_dict())
# net.vgg1.load_state_dict(torch.load("./models/split_vgg16_features_4.pt"))

In [9]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)
# set_trainable(net.mp0,True)
# set_trainable(net.mp1,True)
set_trainable(net.class_predictor,True)
for name,child in net.vgg0.named_children():
    if name[:-1] == "layer":
        [set_trainable(s.ignore_filters,True) for s in child[::2]]
        [set_trainable(s.copy_filters,True) for s in child[::2]]
# for name,child in net.vgg1.named_children():
#     if name[:-1] == "layer":
#         [set_trainable(s.ignore_filters,False) for s in child[::2]]
#         [set_trainable(s.copy_filters,False) for s in child[::2]]


In [10]:
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr = 0.001)
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.01, momentum=0.9)
param_lr = []
# param_lr.append({'params': net.mp0.parameters(),'lr':1e-2,'momentum':0.9})
# param_lr.append({'params': net.mp1.parameters(),'lr':1e-2,'momentum':0.9})
param_lr.append({'params': net.class_predictor.parameters(),'lr':1e-3,'momentum':0.9})

for name,child in net.vgg0.named_children():
    if name[:-1] == "layer":
        for s in child[::2]:
            lr = 0
            if int(name[-1]) > 0:
                lr = 1e-3
            else:
                lr = 0
            param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
            param_lr.append({'params':s.copy_filters.parameters(),'lr':1e-3,'momentum':0.9})
# for name,child in net.vgg1.named_children():
#     if name[:-1] == "layer":
#         for s in child[::2]:
#             lr = 0
#             if int(name[-1]) > 0:
#                 lr = 0
#             else:
#                 lr = 0
#             param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
#             param_lr.append({'params':s.copy_filters.parameters(),'lr':1e-3,'momentum':0.9})

net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-2, eps=1e-08)

30135857 30135857
[{'params': <generator object Module.parameters at 0x7fd43404a620>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17518>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c174c0>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17468>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17410>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17728>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17780>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c177d8>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17830>, 'lr': 0.001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fd435c17888>, 'lr': 0.001, 'momentum': 0.9}, {

In [11]:
# net = nn.DataParallel(net)
net = net.cuda()

In [ ]:
# torch.set_printoptions(threshold=10**6)
iters_per_checkpoint = 60
for epoch in range(10000):  # loop over the dataset multiple times
    running_loss = 0.0
    loss1,loss2 = 0.0,0.0
    acc1,acc2 = 0.0,0.0
    for i,data in enumerate(train_loader,0):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        batch_images = batch_images.cuda(non_blocking=True) 
        batch_impulses = batch_impulses.cuda(non_blocking=True) 
#         batch_gt_responses = batch_gt_responses.cuda(non_blocking=True) 
#         batch_bboxes = batch_bboxes.cuda(non_blocking=True) 
        batch_one_hot = batch_one_hot.cuda(non_blocking=True) 
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_class,pred_masks = net([batch_images,batch_impulses])
        # we are giving no weighting for classes...
        class_loss,mask_loss = model_lib.multi_mask_loss_criterion(pred_class,batch_one_hot,pred_masks,batch_gt_responses,batch_bboxes)
        class_acc, mask_acc = model_lib.accuracy(pred_class,batch_one_hot,pred_masks[1],batch_gt_responses)
        loss = class_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item(); loss1 += class_loss.item(); loss2 += mask_loss.item()
        acc1 += class_acc.item(); acc2 += mask_acc.item()
        if i % iters_per_checkpoint == iters_per_checkpoint-1:
#             scheduler.step((loss1+loss2)/iters_per_checkpoint)
            print("batch: ",i,"epoch: ",epoch, "loss: %0.5f" % (running_loss/iters_per_checkpoint))
            print("class_loss: %0.5f \t mask_loss: %0.5f"%(loss1/iters_per_checkpoint,loss2/iters_per_checkpoint))
            print("class_acc: %0.5f \t mask_acc: %0.5f"%(acc1/iters_per_checkpoint,acc2/iters_per_checkpoint))
#             torch.save(net.state_dict(),model_dir+("model_mask_vgg_%d_%d.pt")%(2,2))
            torch.save(net.state_dict(),model_dir+("bbox_02.pt"))
            running_loss = 0.0; loss1 = 0.0; loss2 = 0.0
            acc1 = 0.0; acc2 = 0.0
    # print("batch: %d time:%0.3f sec" %(i, end-start)); print(loss.item())
print('Finished Training')

batch:  59 epoch:  0 loss: 1.63183
class_loss: 1.63183 	 mask_loss: 1.63183
class_acc: 0.57188 	 mask_acc: 0.57188
batch:  119 epoch:  0 loss: 1.53174
class_loss: 1.53174 	 mask_loss: 1.53174
class_acc: 0.57500 	 mask_acc: 0.57500
batch:  179 epoch:  0 loss: 1.45613
class_loss: 1.45613 	 mask_loss: 1.45613
class_acc: 0.59375 	 mask_acc: 0.59375
batch:  239 epoch:  0 loss: 1.39448
class_loss: 1.39448 	 mask_loss: 1.39448
class_acc: 0.60729 	 mask_acc: 0.60729
batch:  299 epoch:  0 loss: 1.47583
class_loss: 1.47583 	 mask_loss: 1.47583
class_acc: 0.59583 	 mask_acc: 0.59583
batch:  359 epoch:  0 loss: 1.60552
class_loss: 1.60552 	 mask_loss: 1.60552
class_acc: 0.55833 	 mask_acc: 0.55833
batch:  419 epoch:  0 loss: 1.47130
class_loss: 1.47130 	 mask_loss: 1.47130
class_acc: 0.58854 	 mask_acc: 0.58854
batch:  479 epoch:  0 loss: 1.55222
class_loss: 1.55222 	 mask_loss: 1.55222
class_acc: 0.56979 	 mask_acc: 0.56979
batch:  539 epoch:  0 loss: 1.43657
class_loss: 1.43657 	 mask_loss: 1.43

batch:  4319 epoch:  0 loss: 1.19826
class_loss: 1.19826 	 mask_loss: 1.19826
class_acc: 0.68333 	 mask_acc: 0.68333
batch:  4379 epoch:  0 loss: 1.17539
class_loss: 1.17539 	 mask_loss: 1.17539
class_acc: 0.66563 	 mask_acc: 0.66563
batch:  4439 epoch:  0 loss: 1.18518
class_loss: 1.18518 	 mask_loss: 1.18518
class_acc: 0.67708 	 mask_acc: 0.67708
batch:  4499 epoch:  0 loss: 1.17123
class_loss: 1.17123 	 mask_loss: 1.17123
class_acc: 0.66667 	 mask_acc: 0.66667
batch:  4559 epoch:  0 loss: 1.19831
class_loss: 1.19831 	 mask_loss: 1.19831
class_acc: 0.65833 	 mask_acc: 0.65833
batch:  4619 epoch:  0 loss: 1.23574
class_loss: 1.23574 	 mask_loss: 1.23574
class_acc: 0.64792 	 mask_acc: 0.64792
batch:  4679 epoch:  0 loss: 1.16881
class_loss: 1.16881 	 mask_loss: 1.16881
class_acc: 0.67188 	 mask_acc: 0.67188
batch:  4739 epoch:  0 loss: 1.14067
class_loss: 1.14067 	 mask_loss: 1.14067
class_acc: 0.68854 	 mask_acc: 0.68854
batch:  4799 epoch:  0 loss: 1.13895
class_loss: 1.13895 	 mask_

batch:  8579 epoch:  0 loss: 0.91158
class_loss: 0.91158 	 mask_loss: 0.91158
class_acc: 0.74479 	 mask_acc: 0.74479
batch:  8639 epoch:  0 loss: 1.05951
class_loss: 1.05951 	 mask_loss: 1.05951
class_acc: 0.71562 	 mask_acc: 0.71562
batch:  8699 epoch:  0 loss: 0.97307
class_loss: 0.97307 	 mask_loss: 0.97307
class_acc: 0.71562 	 mask_acc: 0.71562
batch:  8759 epoch:  0 loss: 1.02031
class_loss: 1.02031 	 mask_loss: 1.02031
class_acc: 0.71250 	 mask_acc: 0.71250
batch:  8819 epoch:  0 loss: 1.05960
class_loss: 1.05960 	 mask_loss: 1.05960
class_acc: 0.68542 	 mask_acc: 0.68542
batch:  8879 epoch:  0 loss: 0.97111
class_loss: 0.97111 	 mask_loss: 0.97111
class_acc: 0.72188 	 mask_acc: 0.72188
batch:  8939 epoch:  0 loss: 1.01226
class_loss: 1.01226 	 mask_loss: 1.01226
class_acc: 0.72083 	 mask_acc: 0.72083
batch:  8999 epoch:  0 loss: 0.96983
class_loss: 0.96983 	 mask_loss: 0.96983
class_acc: 0.72500 	 mask_acc: 0.72500
batch:  9059 epoch:  0 loss: 0.98380
class_loss: 0.98380 	 mask_

batch:  12779 epoch:  0 loss: 0.86598
class_loss: 0.86598 	 mask_loss: 0.86598
class_acc: 0.74375 	 mask_acc: 0.74375
batch:  12839 epoch:  0 loss: 0.86029
class_loss: 0.86029 	 mask_loss: 0.86029
class_acc: 0.74167 	 mask_acc: 0.74167
batch:  12899 epoch:  0 loss: 0.88388
class_loss: 0.88388 	 mask_loss: 0.88388
class_acc: 0.74792 	 mask_acc: 0.74792
batch:  12959 epoch:  0 loss: 0.81716
class_loss: 0.81716 	 mask_loss: 0.81716
class_acc: 0.75313 	 mask_acc: 0.75313
batch:  13019 epoch:  0 loss: 0.91430
class_loss: 0.91430 	 mask_loss: 0.91430
class_acc: 0.73229 	 mask_acc: 0.73229
batch:  13079 epoch:  0 loss: 0.86214
class_loss: 0.86214 	 mask_loss: 0.86214
class_acc: 0.75208 	 mask_acc: 0.75208
batch:  13139 epoch:  0 loss: 0.88663
class_loss: 0.88663 	 mask_loss: 0.88663
class_acc: 0.74167 	 mask_acc: 0.74167
batch:  13199 epoch:  0 loss: 0.89000
class_loss: 0.89000 	 mask_loss: 0.89000
class_acc: 0.75000 	 mask_acc: 0.75000
batch:  13259 epoch:  0 loss: 0.95774
class_loss: 0.9577

batch:  16979 epoch:  0 loss: 0.78030
class_loss: 0.78030 	 mask_loss: 0.78030
class_acc: 0.77187 	 mask_acc: 0.77187
batch:  17039 epoch:  0 loss: 0.79939
class_loss: 0.79939 	 mask_loss: 0.79939
class_acc: 0.77083 	 mask_acc: 0.77083
